# 把框选出的单词单独保存出来

In [34]:
import cv2
import os
import numpy as np
from PIL import Image

In [38]:
def crop_black_background(image_path):
    """Crop out the black background area from a single image and return the cropped image."""
    # Load image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a binary mask where black regions are 255 and non-black regions are 0
    _, binary_mask = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the bounding box of the largest contour (the black region)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        
        # Crop the image using the bounding box
        cropped_image = image[y:y+h, x:x+w]
        
        return cropped_image
    else:
        return image  # If no contours found, return the original image

def process_images_in_folder(folder_path):
    """Process all images in a folder, crop out the black background, and overwrite the original images."""
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            # Construct full file path
            img_path = os.path.join(folder_path, file_name)
            
            # Process image
            cropped_image = crop_black_background(img_path)
            
            # Save the processed image by overwriting the original image
            cropped_image_pil = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
            cropped_image_pil.save(img_path)
            

In [ ]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL'  # Path to the folder containing images

# Process all images in the folder
process_images_in_folder(folder_path)

In [40]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM'  # Path to the folder containing images

# Process all images in the folder
process_images_in_folder(folder_path)

# 把黑像素点变成白像素点，白像素点变成黑像素点

In [41]:
def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert image from BGR to RGB (OpenCV loads images in BGR format)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Create masks for black and white colors
    mask_black = np.all(image_rgb == [0, 0, 0], axis=-1)
    mask_white = np.all(image_rgb == [255, 255, 255], axis=-1)
    
    # Invert black to white and white to black
    inverted_image = np.where(mask_black[:, :, np.newaxis], [255, 255, 255], image_rgb)
    final_image = np.where(mask_white[:, :, np.newaxis], [0, 0, 0], inverted_image)
    
    # Ensure the image data type is uint8
    final_image = final_image.astype(np.uint8)
    
    # Save the processed image
    cv2.imwrite(image_path, cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR))

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            file_path = os.path.join(folder_path, filename)
            process_image(file_path)

In [42]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL'
process_folder(folder_path)

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [18]:
folder_path = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM'
process_folder(folder_path)

# 把图片截出来

In [25]:
def extract_red_boxes(image):
    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the red color in the HSV space
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for the red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours, mask

def resize_image(image, size=(128, 128)):
    return cv2.resize(image, size)

def crop_words(input_folder, crop_folder):
    if not os.path.exists(crop_folder):
        os.makedirs(crop_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for img_file in image_files:
        img_path = os.path.join(input_folder, img_file)
        image = cv2.imread(img_path)
        
        contours, _ = extract_red_boxes(image)
        
        # Create a subfolder for each image
        img_subfolder = os.path.join(crop_folder, os.path.splitext(img_file)[0])
        if not os.path.exists(img_subfolder):
            os.makedirs(img_subfolder)
        
        for i, cnt in enumerate(contours):
            x, y, w, h = cv2.boundingRect(cnt)
            word_crop = image[y:y+h, x:x+w]
            
            # Save cropped image with a unique name within the subfolder
            crop_path = os.path.join(img_subfolder, f"word_{i}.png")
            cv2.imwrite(crop_path, word_crop)

def resize_and_clean_words(crop_folder):
    if not os.path.exists(crop_folder):
        raise ValueError(f"The folder '{crop_folder}' does not exist.")

    for root, dirs, files in os.walk(crop_folder):
        for img_file in files:
            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, img_file)
                image = cv2.imread(img_path)
                
                # Resize the image
                resized_image = resize_image(image)
                
                # Save the resized image (overwrite the original cropped image)
                cv2.imwrite(img_path, resized_image)

In [28]:
#CVL
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [29]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)

In [23]:
#IAM
input_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM"
crop_folder = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM_rec"

# Step 1: Crop words and save them to crop_folder
crop_words(input_folder, crop_folder)


In [24]:
# Step 2: Resize cropped images and remove red boxes
resize_and_clean_words(crop_folder)

# 把红框去掉

In [30]:
def remove_red_boxes(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the red color in the HSV space
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for the red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Create an image where red pixels are set to white
    white_background = np.ones_like(image) * 255
    no_red_image = np.where(red_mask[:, :, None] == 0, image, white_background)

    return no_red_image

def convert_white_inside_black_outline(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the black color in the HSV space
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])  # Black has very low value in V channel

    # Create a mask for black color
    black_mask = cv2.inRange(hsv, lower_black, upper_black)

    # Find contours in the black mask
    contours, _ = cv2.findContours(black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert image to RGB for easier manipulation
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        roi = image_rgb[y:y+h, x:x+w]
        
        # Create a mask for white pixels inside the bounding box
        white_mask = np.all(roi == [255, 255, 255], axis=-1)
        
        # Convert white pixels to black
        roi[white_mask] = [0, 0, 0]
        
        # Replace the ROI in the original image
        image_rgb[y:y+h, x:x+w] = roi

    # Convert back to BGR for saving
    processed_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    return processed_image

def process_subfolders(root_folder):
    for subdir, _, files in os.walk(root_folder):
        for img_file in files:
            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, img_file)
                image = cv2.imread(img_path)
                
                # Remove red boxes
                image_no_red = remove_red_boxes(image)
                
                # Convert white inside black outline
                processed_image = convert_white_inside_black_outline(image_no_red)
                
                # Save the processed image (overwrite the original image)
                cv2.imwrite(img_path, processed_image)

In [31]:
root_folder = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/CVL_rec'
process_subfolders(root_folder)

KeyboardInterrupt: 

In [ ]:
root_folder = '/root/autodl-tmp/APS360_Project/Machine_Learning_Output/output/IAM_rec'
process_subfolders(root_folder)